In [1]:
import serial

In [74]:
# Good core to start with.
commands = list()
for gcmd in [0, 1, 20, 21, 28, 90, 91]:
    commands.append("G{code}".format(code=gcmd))
for mcmd in [0, 1, 2, 3, 4, 5, 6]:
    commands.append("M{code}".format(code=mcmd))

In [75]:
class GCode(object):
    def __init__(self, *args, **kwargs):
        self.linenumbering = False
        self.buffer = list()
        
    @property
    def code(self):
        if self.linenumbering:
            pass
        return "\n".join(self.buffer)
        
    def save(self, filename):
        pass
    
    def send(self):
        pass
    
    def __str__(self):
        return self.code

    def __repr__(self):
        return "<GCode>[cmds={}]".format(len(self.buffer))
    
    def __iter__(self):
        for buf in self.buffer:
            yield buf
    
    def _repr_html_(self):
        html = list()
        for cmd_line in self.buffer:
            cmd, *args = cmd_line.split(" ")
            html_line = "<b>{cmd}</b> <i>{args}</i>".format(cmd=cmd, args=" ".join(args))
            html.append(html_line)
        return "<br>\n".join(html)
    

def cmd_factory(cmd, doc=None):
    def cmd_fcn(self, **kwargs):
        args = list()
        for key, value in kwargs.items():
            args.append("{key}{value}".format(key=key, value=value))
        
        cmd_str = "{cmd} {args}".format(cmd=cmd, args=" ".join(args))
        self.buffer.append(cmd_str.strip())  
    return cmd_fcn       


for command in commands:
    setattr(GCode, command, cmd_factory(command))

In [76]:
s = serial.Serial(port="/dev/cnc_3018",
                  baudrate=115200,
                  timeout=1)
from time import sleep
sleep(2)
s.flushInput()
sleep(2)

In [77]:
def grbl(cmd):
    s.flushInput()
    s.write("{cmd}\n".format(cmd=cmd).encode())
    responses = s.readlines()
    responses = [response.decode().strip() for response in responses]
    return responses

In [78]:
grbl("$$")

['$0=10',
 '$1=25',
 '$2=0',
 '$3=0',
 '$4=0',
 '$5=0',
 '$6=0',
 '$10=1',
 '$11=0.010',
 '$12=0.002',
 '$13=0',
 '$20=0',
 '$21=0',
 '$22=0',
 '$23=0',
 '$24=25.000',
 '$25=500.000',
 '$26=250',
 '$27=1.000',
 '$30=1000',
 '$31=0',
 '$32=0',
 '$100=250.000',
 '$101=250.000',
 '$102=250.000',
 '$110=500.000',
 '$111=500.000',
 '$112=500.000',
 '$120=10.000',
 '$121=10.000',
 '$122=10.000',
 '$130=200.000',
 '$131=200.000',
 '$132=200.000',
 'ok']

In [79]:
grbl_settings = grbl("$$")

In [81]:
# Quick template for available settings.
print("settings = [")
for grbl_setting in grbl_settings:
    if grbl_setting.startswith("$"):
        key, value = grbl_setting.split("=")
        print("    (\"{}\", \"\"),".format(key))
print("    ]")

settings = [
    ("$0", ""),
    ("$1", ""),
    ("$2", ""),
    ("$3", ""),
    ("$4", ""),
    ("$5", ""),
    ("$6", ""),
    ("$10", ""),
    ("$11", ""),
    ("$12", ""),
    ("$13", ""),
    ("$20", ""),
    ("$21", ""),
    ("$22", ""),
    ("$23", ""),
    ("$24", ""),
    ("$25", ""),
    ("$26", ""),
    ("$27", ""),
    ("$30", ""),
    ("$31", ""),
    ("$32", ""),
    ("$100", ""),
    ("$101", ""),
    ("$102", ""),
    ("$110", ""),
    ("$111", ""),
    ("$112", ""),
    ("$120", ""),
    ("$121", ""),
    ("$122", ""),
    ("$130", ""),
    ("$131", ""),
    ("$132", ""),
    ]


In [69]:
settings = [
    ("$0", "step_pulse"),
    ("$1", "step_idle_delay"),
    ("$2", "step_port_invert"),
    ("$3", "direction_port_invert"),
    ("$4", "step_enable_invert"),
    ("$5", "limit_pin_invert"),
    ("$6", "probe_pin_invert"),
    ("$10", "status_report"),
    ("$11", "junction_deviation"),
    ("$12", "arc_tolerance"),
    ("$13", "report_inches"),
    ("$20", "soft_limits"),
    ("$21", "hard_limits"),
    ("$22", "homing_cycle"),
    ("$23", "homing_dir_invert"),
    ("$24", "homing_feed"),
    ("$25", "homing_seek"),
    ("$26", "homing_debounce"),
    ("$27", "homing_pull_off"),
    ("$30", "max_spindle_speed"),
    ("$31", "min_spindle_speed"),
    ("$32", "laser_mode"),
    ("$100", "x_steps_mm"),
    ("$101", "y_steps_mm"),
    ("$102", "z_steps_mm"),
    ("$110", "x_max_rate"),
    ("$111", "y_max_rate"),
    ("$112", "z_max_rate"),
    ("$120", "x_acceleration"),
    ("$121", "y_acceleration"),
    ("$122", "z_acceleration"),
    ("$130", "x_travel"),
    ("$131", "y_travel"),
    ("$132", "z_travel"),
    ]

In [92]:
def grbl_getter_generator(cmd):
    def grbl_getter(self):
        pass
    return grbl_getter
    
def grbl_setter_generator(cmd):
    def grbl_setter(self, value):
        ret = grbl("{cmd}={value}".format(cmd=cmd, value=value))
        return ret
    return grbl_setter

In [93]:
for setting in settings:
    print(setting)
    break

('$0', 'step_pulse')


In [94]:
grbl("$0=10")

['ok']

In [95]:
step_pulse = grbl_setter_generator("$0")

In [96]:
step_pulse(_, 10)

['ok']

In [99]:
config = grbl("$$")

In [101]:
for config_line in config:
    print(config_line)
    break

$0=10


In [106]:
if config_line.startswith("$"):
    key, value = config_line.split("=")

In [103]:
key

'$0'

In [104]:
value

'10'

In [108]:
cmd = "$0"

In [112]:
def grbl_getter_generator(cmd):
    def grbl_getter(self):
        config = grbl("$$")
        for config_line in config:
            if config_line.startswith("$"):
                key, value = config_line.split("=")
                if key == cmd:
                    return float(value)
        return None
    return grbl_getter

In [113]:
step_pulse = grbl_getter_generator("$0")

In [115]:
step_pulse(_)

10.0